## Quroa Kaggle Challenge

First Attempt at the Quora kaggle challenge.  Use just one word embedding and a simple CNN model,

For each qid in the test set, you must predict whether the corresponding question_text is insincere (1) or not (0). Predictions should only be the integers 0 or 1.



- https://www.kaggle.com/c/quora-insincere-questions-classification

In [90]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate

SEED = 2018
import tensorflow as tf
np.random.seed(SEED)
tf.set_random_seed(SEED)

from tqdm import tqdm
tqdm.pandas()

import gc,re

import warnings
warnings.filterwarnings('ignore')


In [3]:
#do a clean up
#del word_index, embeddings_index, all_embs, embedding_matrix, model
#import gc; gc.collect()
#time.sleep(10)

In [4]:
#read in the data
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


In [5]:
#lets look at the data
train_df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [13]:
def clean_text(x):
    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'isnt':'is not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium'
                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [14]:
# Clean the text
train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: clean_text(x))
test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: clean_text(x))

# Clean numbers
train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: clean_numbers(x))
test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: clean_numbers(x))

# Clean speelings
train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: replace_typical_misspell(x))



  0%|                                                                                      | 0/1175509 [00:00<?, ?it/s]

  1%|▊                                                                     | 12978/1175509 [00:00<00:09, 121843.71it/s]

  3%|█▊                                                                    | 30738/1175509 [00:00<00:08, 131025.48it/s]

  4%|██▌                                                                   | 43211/1175509 [00:00<00:08, 128970.43it/s]

  5%|███▎                                                                  | 56106/1175509 [00:00<00:08, 128490.52it/s]

  6%|████▎                                                                 | 72248/1175509 [00:00<00:08, 136402.84it/s]

  8%|█████▍                                                                | 90630/1175509 [00:00<00:07, 143017.62it/s]

  9%|██████▎                                                              | 107104/1175509 [00:00<00:07, 148026.40it/s]

 11%|███████▎                 

 91%|██████████████████████████████████████████████████████████████      | 1072029/1175509 [00:06<00:00, 155504.76it/s]

 93%|██████████████████████████████████████████████████████████████▉     | 1087836/1175509 [00:06<00:00, 155925.57it/s]

 94%|███████████████████████████████████████████████████████████████▉    | 1105443/1175509 [00:07<00:00, 156358.55it/s]

 95%|████████████████████████████████████████████████████████████████▊   | 1121082/1175509 [00:07<00:00, 155501.27it/s]

 97%|█████████████████████████████████████████████████████████████████▊  | 1136832/1175509 [00:07<00:00, 155756.07it/s]

 98%|██████████████████████████████████████████████████████████████████▋ | 1152609/1175509 [00:07<00:00, 156031.04it/s]

 99%|███████████████████████████████████████████████████████████████████▌| 1168390/1175509 [00:07<00:00, 156206.73it/s]

100%|████████████████████████████████████████████████████████████████████| 1175509/1175509 [00:07<00:00, 157233.72it/s]

  0%|                           

 61%|██████████████████████████████████████████▎                          | 721379/1175509 [00:05<00:03, 130646.99it/s]

 62%|███████████████████████████████████████████                          | 734445/1175509 [00:05<00:03, 128522.70it/s]

 64%|███████████████████████████████████████████▊                         | 747306/1175509 [00:05<00:03, 127142.57it/s]

 65%|████████████████████████████████████████████▋                        | 761405/1175509 [00:05<00:03, 127232.32it/s]

 66%|█████████████████████████████████████████████▌                       | 776319/1175509 [00:06<00:03, 128462.88it/s]

 67%|██████████████████████████████████████████████▎                      | 789485/1175509 [00:06<00:02, 129257.15it/s]

 68%|███████████████████████████████████████████████                      | 802810/1175509 [00:06<00:02, 129859.91it/s]

 70%|████████████████████████████████████████████████                     | 817959/1175509 [00:06<00:02, 130361.44it/s]

 71%|███████████████████████████

 68%|██████████████████████████████████████████████▋                      | 796313/1175509 [00:02<00:01, 295071.98it/s]

 70%|████████████████████████████████████████████████▍                    | 825824/1175509 [00:02<00:01, 293004.48it/s]

 73%|██████████████████████████████████████████████████▏                  | 855131/1175509 [00:02<00:01, 291512.78it/s]

 76%|████████████████████████████████████████████████████▏                | 888444/1175509 [00:03<00:00, 291574.93it/s]

 78%|██████████████████████████████████████████████████████               | 921791/1175509 [00:03<00:00, 293382.10it/s]

 81%|███████████████████████████████████████████████████████▊             | 951739/1175509 [00:03<00:00, 294191.64it/s]

 84%|█████████████████████████████████████████████████████████▋           | 982156/1175509 [00:03<00:00, 294977.00it/s]

 86%|██████████████████████████████████████████████████████████▊         | 1015890/1175509 [00:03<00:00, 295303.11it/s]

 89%|███████████████████████████

In [16]:
train_df.head()

,qid,question_text,target
651064,7f8590ef60e30b4344fd,What have been the best exhibits at the Museo ...,0
1294259,fda9538a2e0a5b2dfc3c,How can I rotate batch image files,0
1205706,ec528b4e3abc3347cd21,Which is the best cable operator in Thane west...,0
460090,5a1a41ea2086f2264eab,How do I expand factor and simplify in algebra,0
1277213,fa4f394af94b2b094e15,Do you judge people often,0


C:\Users\richard\Anaconda3\envs\tf15\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


Next steps are as follows:
 * Split the training dataset into train and val sample - cross val too expensive
 * Fill up the missing values in the text column with '_na_'
 * Tokenize the text column and convert them to vector sequences
 * Pad the sequence as needed - if the number of words in the text is greater than 'max_len' trunacate them to 'max_len' or if the number of words in the text is lesser than 'max_len' add zeros for remaining values.

In [20]:
## split to train and val
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)

## some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

## fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

In [21]:
## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
#produce a list of lists- each list is a integer representation of each word in the sentence
train_X = tokenizer.texts_to_sequences(train_X)
#print((train_X[:10]))


val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values

C:\Users\richard\Anaconda3\envs\tf15\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [22]:
EMBEDDING_FILE = 'D:\\ml_code\\embeddings\\glove.840B.300d\\glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE,errors='ignore'))


C:\Users\richard\Anaconda3\envs\tf15\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [24]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
print(nb_words)

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

50000


In [25]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total para

In [26]:
#run for just one epoch
model.fit(train_X, train_y, batch_size=512, epochs=3, validation_data=(val_X, val_y),verbose=2)

Train on 952162 samples, validate on 105796 samples
Epoch 1/3
 - 122s - loss: 0.1129 - acc: 0.9562 - val_loss: 0.1038 - val_acc: 0.9595
Epoch 2/3
 - 120s - loss: 0.0931 - acc: 0.9631 - val_loss: 0.1027 - val_acc: 0.9592
Epoch 3/3
 - 119s - loss: 0.0788 - acc: 0.9687 - val_loss: 0.1071 - val_acc: 0.9589


In [27]:
#apply to validation set
pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)

105796/105796 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 3s 33us/step


In [28]:
#look for a better threshold
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.6020408163265306
F1 score at threshold 0.11 is 0.6091571949388805
F1 score at threshold 0.12 is 0.6152756591850076
F1 score at threshold 0.13 is 0.6209873810050919
F1 score at threshold 0.14 is 0.6259627818069377
F1 score at threshold 0.15 is 0.6313211845102505
F1 score at threshold 0.16 is 0.6348803689824157
F1 score at threshold 0.17 is 0.6397802326261033
F1 score at threshold 0.18 is 0.6427811280595956
F1 score at threshold 0.19 is 0.6461759253722419
F1 score at threshold 0.2 is 0.648821752265861
F1 score at threshold 0.21 is 0.650634765625
F1 score at threshold 0.22 is 0.6528874629812439
F1 score at threshold 0.23 is 0.6544434050514499
F1 score at threshold 0.24 is 0.6552613371910182
F1 score at threshold 0.25 is 0.6566842205443817
F1 score at threshold 0.26 is 0.6591841970241149
F1 score at threshold 0.27 is 0.6601991980338896
F1 score at threshold 0.28 is 0.6608797807074794
F1 score at threshold 0.29 is 0.6622760800842993
F1 score at threshold 0.3 i

In [29]:
pred_glove_test_y = model.predict([test_X], batch_size=1024, verbose=1)

56370/56370 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 2s 33us/step


In [30]:
pred_glove_test_y

array([[8.9275418e-04],
       [1.9369295e-04],
       [1.9134153e-05],
       ...,
       [3.1709255e-04],
       [2.6350641e-03],
       [8.4229267e-01]], dtype=float32)

In [31]:
#do a clean up
del word_index, embeddings_index, all_embs, embedding_matrix, model
import gc; gc.collect()
time.sleep(10)

**Wiki News FastText Embeddings:**

Now let us use the FastText embeddings trained on Wiki News corpus in place of Glove embeddings and rebuild the model.

In [32]:
EMBEDDING_FILE = 'D:\\ml_code\\embeddings\\wiki-news-300d-1M\\wiki-news-300d-1M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE,errors='ignore') if len(o)>100)


In [33]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]
print(all_embs.shape)

(999856, 300)


In [34]:
word_index = tokenizer.word_index
print(len(word_index))
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
print(embedding_matrix.shape)

165640
(50000, 300)


In [35]:
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

#len(word_index.items())
#print(len(embedding_vector))

In [52]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_6 (Bidirection (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 16)                2064      
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 17        
Total para

In [53]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y),verbose=2)

Train on 952162 samples, validate on 105796 samples
Epoch 1/2
 - 121s - loss: 0.1154 - acc: 0.9556 - val_loss: 0.1054 - val_acc: 0.9585
Epoch 2/2
 - 120s - loss: 0.0928 - acc: 0.9629 - val_loss: 0.1071 - val_acc: 0.9591


In [54]:
pred_fasttext_val_y = model.predict([val_X], batch_size=1024, verbose=1)

105796/105796 [==============================] - ETA: 17 - ETA: 8 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 4s 35us/step


In [55]:
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_fasttext_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.606289037260392
F1 score at threshold 0.11 is 0.6121956569423119
F1 score at threshold 0.12 is 0.6185348434971824
F1 score at threshold 0.13 is 0.6239918209701237
F1 score at threshold 0.14 is 0.6302065305180571
F1 score at threshold 0.15 is 0.6353684949950242
F1 score at threshold 0.16 is 0.6399002197541129
F1 score at threshold 0.17 is 0.6427282569469506
F1 score at threshold 0.18 is 0.6464855646241929
F1 score at threshold 0.19 is 0.6484788767089544
F1 score at threshold 0.2 is 0.6519461077844312
F1 score at threshold 0.21 is 0.6541879372282094
F1 score at threshold 0.22 is 0.6566345724670026
F1 score at threshold 0.23 is 0.6594072164948453
F1 score at threshold 0.24 is 0.6612388458281769
F1 score at threshold 0.25 is 0.6632894736842105
F1 score at threshold 0.26 is 0.6652505808164619
F1 score at threshold 0.27 is 0.6673369528788793
F1 score at threshold 0.28 is 0.6679740329997296
F1 score at threshold 0.29 is 0.6708099986340663
F1 score at threshold 0

In [58]:
pred_fasttext_test_y = model.predict([test_X], batch_size=1024, verbose=1)

56370/56370 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 2s 34us/step


In [59]:
del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(10)

**Paragram Embeddings:**

In this section, we can use the paragram embeddings and build the model and make predictions.

In [80]:
EMBEDDING_FILE = 'D:\\ml_code\\embeddings\\paragram_300_sl999\\paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

In [83]:

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        

In [85]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(16, activation="relu")(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 100)               0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 128)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_13 (Dropout)         (None, 32)                0         
__________

In [78]:
print(max_features)
print(embed_size)
print(maxlen)

50000
300
100


In [86]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y),verbose=2)

Train on 952162 samples, validate on 105796 samples
Epoch 1/2
 - 122s - loss: 0.1157 - acc: 0.9549 - val_loss: 0.1048 - val_acc: 0.9591
Epoch 2/2
 - 122s - loss: 0.0955 - acc: 0.9618 - val_loss: 0.1054 - val_acc: 0.9593


In [87]:
pred_paragram_val_y = model.predict([val_X], batch_size=1024, verbose=1)

105796/105796 [==============================] - ETA: 25 - ETA: 10 - ETA: 7 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 4s 35us/step


In [88]:
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_paragram_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.6375881240327851
F1 score at threshold 0.11 is 0.6424065420560747
F1 score at threshold 0.12 is 0.6477333491771138
F1 score at threshold 0.13 is 0.6519738435456527
F1 score at threshold 0.14 is 0.6544316228823963
F1 score at threshold 0.15 is 0.6583130727510753
F1 score at threshold 0.16 is 0.6626627892274624
F1 score at threshold 0.17 is 0.6657252888318357
F1 score at threshold 0.18 is 0.6666233597505522
F1 score at threshold 0.19 is 0.6682888333552546
F1 score at threshold 0.2 is 0.6703026272031926
F1 score at threshold 0.21 is 0.6706135629709364
F1 score at threshold 0.22 is 0.6728298552103869
F1 score at threshold 0.23 is 0.6736235067966497
F1 score at threshold 0.24 is 0.6750346740638002
F1 score at threshold 0.25 is 0.6752136752136753
F1 score at threshold 0.26 is 0.676314860571832
F1 score at threshold 0.27 is 0.6770833333333334
F1 score at threshold 0.28 is 0.6774634708126395
F1 score at threshold 0.29 is 0.6774123284687432
F1 score at threshold 0

In [65]:
pred_paragram_test_y = model.predict([test_X], batch_size=1024, verbose=1)

56370/56370 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 2s 33us/step


In [66]:
#del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
#import gc; gc.collect()
#time.sleep(10)

In [69]:
pred_val_y = 0.33*pred_glove_val_y + 0.33*pred_fasttext_val_y + 0.33*pred_paragram_val_y 

In [70]:
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_val_y>thresh).astype(int))))

F1 score at threshold 0.1 is 0.5997540731632339
F1 score at threshold 0.11 is 0.6082183186951067
F1 score at threshold 0.12 is 0.6158604874406698
F1 score at threshold 0.13 is 0.6229971212861876
F1 score at threshold 0.14 is 0.6282242474454571
F1 score at threshold 0.15 is 0.6337822671156004
F1 score at threshold 0.16 is 0.640269298796143
F1 score at threshold 0.17 is 0.6449855072463769
F1 score at threshold 0.18 is 0.6496762801648028
F1 score at threshold 0.19 is 0.6533627211532733
F1 score at threshold 0.2 is 0.6573460286850669
F1 score at threshold 0.21 is 0.6607317073170732
F1 score at threshold 0.22 is 0.6644051070128908
F1 score at threshold 0.23 is 0.6677059300367899
F1 score at threshold 0.24 is 0.6705333501450007
F1 score at threshold 0.25 is 0.6733443919867297
F1 score at threshold 0.26 is 0.674739147236893
F1 score at threshold 0.27 is 0.6765412849996751
F1 score at threshold 0.28 is 0.6761579845164677
F1 score at threshold 0.29 is 0.6773167174864113
F1 score at threshold 0.

In [72]:
pred_test_y = 0.33*pred_glove_test_y + 0.33*pred_fasttext_test_y + 0.33*pred_paragram_test_y
pred_test_y = (pred_test_y>0.37).astype(int)

In [73]:
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)